In [1]:
# ============================================================
# ✅ SETUP
# ============================================================

!git clone https://github.com/NVlabs/edm2.git /kaggle/working/edm2
%cd /kaggle/working/edm2
!pip install click tqdm psutil scipy pillow --quiet
%cd /kaggle/working/edm2


Cloning into '/kaggle/working/edm2'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 60 (delta 13), reused 10 (delta 10), pack-reused 33 (from 1)
Receiving objects: 100% (60/60), 1.27 MiB | 9.80 MiB/s, done.
Resolving deltas: 100% (24/24), done.
/kaggle/working/edm2
/kaggle/working/edm2


In [2]:
# ============================================================
# ✅ PATCH: LINEAR TRAINING NOISE SCHEDULE
# ============================================================
import torch
import training.training_loop as loop

def linear_sigma(batch_size, device, sigma_min=0.002, sigma_max=80):
    """
    Sample sigma linearly between sigma_max → sigma_min.
    t ~ U[0,1]
    sigma(t) = sigma_max - (sigma_max - sigma_min) * t
    """
    t = torch.rand(batch_size, 1, 1, 1, device=device)
    return sigma_max - (sigma_max - sigma_min) * t


def patched_call(self, net, images, labels=None):
    """
    This replaces EDM2Loss.__call__ entirely.
    Training noise schedule is now linear, not log-normal.
    """
    batch = images.shape[0]

    # === LINEAR NOISE SCHEDULE ===
    sigma = linear_sigma(batch, images.device)

    # === EDM2 original weighting formula ===
    weight = (sigma**2 + self.sigma_data**2) / (sigma * self.sigma_data)**2

    # Add noise
    noise = torch.randn_like(images) * sigma

    # Forward UNet pass
    denoised, logvar = net(images + noise, sigma, labels, return_logvar=True)

    # Compute loss
    loss = (weight / logvar.exp()) * ((denoised - images)**2) + logvar
    return loss


# Inject the patch
loop.EDM2Loss.__call__ = patched_call

print("✅ Linear training schedule injected successfully!")


✅ Linear training schedule injected successfully!


In [3]:
!torchrun --standalone --nproc_per_node=2 train_edm2.py \
    --outdir=/kaggle/working/training-runs/celeba64-edm2-linear-noise-inject2 \
    --data=/kaggle/input/celeva-64x64-dataset/celeba64/train \
    --cond=False \
    --preset=edm2-img64-xs \
    --batch=64 \
    --batch-gpu=32 \
    --duration=2Mi \
    --status=16Ki \
    --snapshot=256Ki \
    --checkpoint=0 \
    --seed=0


W1114 02:56:46.150000 54 torch/distributed/run.py:792] 
W1114 02:56:46.150000 54 torch/distributed/run.py:792] *****************************************
W1114 02:56:46.150000 54 torch/distributed/run.py:792] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1114 02:56:46.150000 54 torch/distributed/run.py:792] *****************************************
[W1114 02:56:56.462589849 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W1114 02:57:06.473134930 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W1114 02:57:18.251279316 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W1114 02:57:18.263149146 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W1114 02:57:28.261795933 socket.cpp:204]

In [4]:
!tail -n 20 /kaggle/working/training-runs/celeba64-edm2-linear/log.txt

tail: cannot open '/kaggle/working/training-runs/celeba64-edm2-linear/log.txt' for reading: No such file or directory
